# Panda data and EDA
Exploratory Data Analysis

In [ ]:
is_teacher_deck = True

if is_teacher_deck:
    support_folder = '/content/drive/MyDrive/Classroom/high_school_programming/notebooks/materials/'
else:
    support_folder =  '/content/drive'

from google.colab import drive
drive.mount('/content/drive')

from IPython.display import Image

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Before we start -- ML 101

<font size=4 color=blue>Steps for Machine Learning Project</font><br>
<i>conventionaly, all follow these steps, more or less.</i><br><br>
**<font color=green><u>Gathering data**</u></font><br><br>
**<font color=green><u>Exploratory Data Analysis (EDA)**</u></font><br><br>
**Preprocessing data**<br><br>
**Build/Test Model**<br><br>
**Prediction** <br>

# Gathering data

1. Revisit: Libraries -- pandas, <s>alpaca</s>, yfinance, matplotlib
2. Get historical data using yfinance
3. Store data

## 1. Revisit: Libraries
**pandas, alpaca, yfinance, matplotlib**
* How about yfinance? Don't need an account. yfinance does not provide real-time data streaming capabilities. It is more focused on retrieving historical data for analysis rather than providing live or real-time data updates

* We will use yfinance for analysis

## 2. yfinance: check and prepare

**Let take these steps:**
### a). check yfinace apis
### b). decide what we need
### c). try one manually
### d). write functions



### **a). check yfinace apis**


In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
!pip install requests_cache

In [ ]:
!pip install requests-ratelimiter

In [ ]:
# load libraries
import yfinance as yf
from datetime import datetime, timedelta
import time
import os
import json
from bs4 import BeautifulSoup
import pandas as pd
import csv
import numpy as np

* check  [yfinance in Pypi](https://pypi.org/project/yfinance/)

* use <font color=green>help</font>

In [ ]:
help(yf)

### **b). decide what we need**

In [ ]:
tsla = yf.Ticker("TSLA")
# get all stock info
tsla.info

In [ ]:
# get historical market data
hist = tsla.history(period="1mo")
hist

In [ ]:
# periods: (['1d',  '5d',  '1mo',  '3mo',  '6mo',  '1y',  '2y',  '5y',  '10y',  'ytd',  'max'])
# interval: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
# 1m data is only retrievable for the last 7 days,
# intraday (interval <1d) only for the last 60 days

tsla.history_metadata

In [ ]:
# show actions (dividends, splits, capital gains)
tsla.actions

In [ ]:
tsla.dividends

Series([], Name: Dividends, dtype: float64)

In [ ]:
tsla.splits

Series([], Name: Stock Splits, dtype: float64)

In [ ]:
tsla.capital_gains

[]

In [ ]:
# show share count
tsla.get_shares_full(start='2021-06-29', end=None)

In [ ]:
news = tsla.news
news[0]

In [ ]:
# Define the stock symbol and the date range
symbol = "tsla"
end_date = datetime.today()
start_date = end_date - timedelta(days=2*365)

# Download the stock data
df_stock_hitory = yf.download(symbol, start=start_date, end=end_date)

df_stock_hitory.head(1)


<font size=4 color=blue>many more!</font>

In [ ]:
if 1 == 11:
    # show financials:
    tsla.income_stmt
    tsla.quarterly_income_stmt
    # - balance sheet
    tsla.balance_sheet
    tsla.quarterly_balance_sheet
    # - cash flow statement
    tsla.cashflow
    tsla.quarterly_cashflow
    # show holders
    tsla.major_holders
    tsla.institutional_holders
    tsla.mutualfund_holders
    tsla.earnings_dates
    tsla.isin
    # show options expirations
    tsla.options
    # get option chain for specific expiration
    opt = tsla.option_chain('YYYY-MM-DD')
    # data available via: opt.calls, opt.puts

<font size=3 color=red> We choose: </font>
* historical data
* news  (later)

**what intervals and periods**
* available periods: (['1d',  '5d',  '1mo',  '3mo',  '6mo',  '1y',  '2y',  '5y',  '10y', 'ytd',  'max'])
* available intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

**for this project:**
* time periods 1d, 5d, 1y
* interval: 1m, 5m, 1d

### c). try one manually

* Try AAPL

In [ ]:
apple = yf.Ticker("aapl")
# get all stock info
apple.info

**down 1m interval data from past 1 d**
* not to include before/after hour *transactions*

In [ ]:
yf.download(tickers = 'aapl',  # list of tickers
            period = "1d",         # time period
            interval = "1m",       # trading interval
            prepost = False,       # download pre/post market hours or not
            repair = True)         # repair obvious price errors

* what about time range

In [ ]:
yf.download(tickers = 'aapl',  # list of tickers
            start = "2023-06-01",         # from
            end =  "2023-07-01",       # to
            prepost = False,       # download pre/post market hours or not
            repair = True)

* what about try download two stocks in one call?
    - add DASH -- Doordask

In [ ]:
yf.download(tickers = "DASH AAPL",
            period = "1mo",
            interval = "1d",
            prepost = False,
            repair = True)

### d). decide the list

**Something to remember and put to practice**
* Plan what it does: input and output. Following SRP - single Responsibility Principle
* Give a concise and meaningful name. low_case_with_underscore.
* Use docstring right after function signature
* Meaningful parameter names. Not too many. If need many, consider use dictionary
* Optional: type hints for parameters and output  
* Minimize use of global parameters.   
* Follow DRY! Don't Repeat Yourself. Mimimum duplicate code within or across  
* More on PEP 8 style guide (Python Enhancement Proposal)

**<font color=blue>Describe our goals</font>**

* We need to load historical trading data for a number of stocker tickers(symbols).
* We need diffent interval and periods.
* We need news data (later)
* We need to save data

**Tickers**
* two full list of tickers for nyse and nasdaq are directly stripped from www and saved to github <br>

<br><font color=blue><u>raw list of tickers</u></font>
* url_nyse = 'https://raw.githubusercontent.com/stempro/stockai/stockai/data/nyse20230701.csv'
* url_nasdaq = 'https://raw.githubusercontent.com/stempro/stockai/stockai/data/nasdaq20230701.csv'

<br><font color=blue><u>valid list of tickers with basic info</u></font>
* url_nyse_info = 'https://raw.githubusercontent.com/stempro/stockai/stockai/data/NYSE_ticker_info.csv'
* url_nasdaq_info = 'https://raw.githubusercontent.com/stempro/stockai/stockai/data/NASDAQ_ticker_info.csv'


**Decide a list of companies/industries/sections**
* lets focus on high tech


In [ ]:
url_nyse_info = 'https://raw.githubusercontent.com/stempro/stockai/stockai/data/NYSE_ticker_info.csv'
url_nasdaq_info = 'https://raw.githubusercontent.com/stempro/stockai/stockai/data/NASDAQ_ticker_info.csv'

def choose_(industries, sectors, marketcap_from, marketcap_to,
           averageVolume_from, averageVolume_to, limit_n):

    df_nyse = pd.read_csv(url_nyse_info)
    df_nasdaq = pd.read_csv(url_nasdaq_info)
    df_total = pd.concat([df_nyse, df_nasdaq])
    df_ = df_total.copy()
    #advanced: df_['industry'].str.lower().isin([x.lower() for x in industries])
    if industries:
        df_ = df_[df_['industry'].isin(industries)]
        if df_.shape[0]<=limit_n:
            return df_, df_total

    if sectors:
        df_ = df_[df_['sector'].isin(sectors)]
        if df_.shape[0]<=limit_n:
            return df_, df_total

    df_ = df_[df_['marketCap'].between(marketcap_from, marketcap_to)]
    if df_.shape[0]<=limit_n:
        return df_, df_total

    df_ = df_[df_['averageVolume'].between(averageVolume_from, averageVolume_to)]
    if df_.shape[0]<=limit_n:
        return df_, df_total
    else:
        return df_.sample(limit_n), df_total

def choose_and_save_my_list(extras=[], industries=[], sectors=[],
        marketcap_from=-np.inf, marketcap_to=np.inf,
        averageVolume_from=-np.inf, averageVolume_to=np.inf, limit_n = 50,
        refresh_list=False):

    your_project_file = f"{support_folder}/Project_ticker_list.csv"

    if os.path.exists(your_project_file) and (not refresh_list):
        return pd.read_csv(your_project_file)

    df_, df_total = choose_(industries=industries, sectors=sectors,
        marketcap_from=marketcap_from, marketcap_to=marketcap_to,
        averageVolume_from=averageVolume_from,
        averageVolume_to=averageVolume_to, limit_n = limit_n)

    if extras:
        extras = [ticker for ticker in extras if ticker not in list(df_["Ticker"])]
        df_ = pd.concat([df_, df_total[df_total["Ticker"].isin(extras)]])
    # save to YOUR google drive
    df_.to_csv(your_project_file)

    return df_


In [ ]:
df_= choose_and_save_my_list(extras=["AAPL", "TSLA", "META"],
                        sectors=['Technology','Communication Services'] ,
                        marketcap_from=2.50e10 )

In [ ]:
df_.head(2)

,Unnamed: 0,Ticker,industry,sector,averageVolume,averageVolume10days,marketCap
0,35,LRCX,Semiconductor Equipment & Materials,Technology,1336162,1110420,87453999104
1,96,DDOG,Software—Application,Technology,4924927,3426510,31637094400


## yfinance: get data for MY app
** recall**:

---

    * what intervals and periods   
        * available periods: (['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max'])
    
    * available intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
        * for this project:

 **My App:**
    <br><b><u>time periods 1d, 5d, 1y</b></u>
    <br><b><u>interval: 1m, 5m, 1d</b></u>




### a). Get the ticker list
* file path: your_project_file = f"{support_folder}/Project_ticker_list.csv"

* https://github.com/ranaroussi/yfinance/wiki/Tickers

In [ ]:
your_project_file = f"{support_folder}/Project_ticker_list.csv"
df_tickers = pd.read_csv(your_project_file)

### b). Wait: should we track?
* use a json file track what has already been downloaded.
* why? yfinance is the opensource and community managed. not so reliable

In [ ]:
def track_progress(jsonobj=None, action="load"):
    """
        action: load, dump
        to simplify, we use all fix file name and schema.
    """
    track_json_file = f"{support_folder}/yf_progress.json"
    if action.lower() == "load":
        if os.path.exists(track_json_file):
            return json.load(open(track_json_file, 'r'))
        else:
            return {}
    else:
        if not jsonobj:
            return
        json.dump(jsonobj, open(track_json_file, 'w'))

### c). download

In [ ]:
def get_histories(intervals=["1m","5m","1d"],
                  max_tickers_per_call = 100,
                  reload=True):
    """
        action: load, dump
        to simplify, we use all fix file name and folder name.
        time periods 1d, 5d, 1y
        interval: 1m, 5m, 1d. for 1m and 5m, we take 5 days of data
            for 1d. with take 1 y
        filename convention: interval_period_yyyymmmddd.csv
        progress_json:
            1m: True
    """
    progress_json = track_progress(action="load")

    data_folder = f"{support_folder}/data"
    if not os.path.exists(data_folder):
        os.mkdir(data_folder)

    ticker_df = pd.read_csv(your_project_file)
    tickers = list(ticker_df['Ticker'])[:10]
    n_tickers = len(tickers)
    datetime_str = datetime.strftime(datetime.now(), "%YYYY%m%d%H%M")

    for interval in intervals:
        if (not reload) and interval in progress_json:
            continue
        period = "5d" if interval in ["1m", "5m"] else "1y"
        for i in range(0, n_tickers, max_tickers_per_call):

            csv_file = f"{data_folder}/{period}_{interval}_{datetime_str}_{i}.csv"

            tickers_ = tickers[i:(i+max_tickers_per_call)]

            df_ = yf.download(tickers_, interval=interval, period=period, threads=True,
                              prepost = False, repair = True)
            df_ = df_.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index(level=1)
            df_ = df_.reset_index()
            if df_.shape[0]>0:
                df_.to_csv(csv_file)
                progress_json[interval] = "Downloaded"
                #save it right away
                track_progress(jsonobj=progress_json, action="save")

    return progress_json


In [ ]:
get_histories()

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed


{'1m': 'Downloaded', '5m': 'Downloaded', '1d': 'Downloaded'}

## data: pandas
* The name "pandas" is derived from the term "panel data," which is an econometrics term
* let read --> peek/maninpuate/analyze --> output

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

**read to pandas**
* Sum up: we already know we can read directly from url
* can read from hard drive as well

**where are our data**
* support_folder
* data_folder


In [290]:
data_folder = f"{support_folder}/data"
files = os.listdir(data_folder)

**use one df for how to check pandas dataframe**

In [291]:
files

['5d_1m_2023YYY07041922_0.csv',
 '5d_5m_2023YYY07041922_0.csv',
 '1y_1d_2023YYY07041922_0.csv']

In [292]:
df_sample = pd.read_csv(data_folder+'/'+files[2])

In [293]:
df_sample

,Unnamed: 0,Date,Ticker,Adj Close,Close,High,Low,Open,Repaired?,Volume
0,0,2022-07-05,ADBE,376.49,376.49,378.52,359.82,363.50,False,3120500
1,1,2022-07-05,AMX,19.67,20.15,20.28,19.78,20.11,False,1403900
2,2,2022-07-05,DDOG,108.38,108.38,110.08,98.06,100.92,False,6543700
3,3,2022-07-05,IBM,130.89,137.62,139.97,135.27,139.97,False,6273400
4,4,2022-07-05,INFY,18.22,18.67,18.69,18.36,18.51,False,10265200
...,...,...,...,...,...,...,...,...,...,...
2505,2505,2023-07-03,LRCX,650.99,650.99,651.01,638.85,642.86,False,621900
2506,2506,2023-07-03,NOW,562.87,562.87,564.56,555.94,560.85,False,473400
2507,2507,2023-07-03,ORCL,117.15,117.15,118.25,116.83,118.01,False,4284700
2508,2508,2023-07-03,RBLX,42.10,42.10,42.18,40.20,40.32,False,5251700


**Peek, manipulate and analysis**

Pandas is one of the python developers' favorites. We need get some handon only on:
* peek: head, tail, sample. take a look
* inspect: info, summary, shape, columns, dtypes
* loc, iloc, filter
* analysis: agg, groupby, change



In [275]:
df_sample.shape

(3530, 10)

In [276]:
df_sample.describe()

,Unnamed: 0,Adj Close,Close,High,Low,Open,Volume
count,3530.00,3520.00,3520.00,3520.00,3520.00,3520.00,3530.00
mean,1764.50,211.44,211.44,211.66,211.18,211.43,55400.13
std,1019.17,234.08,234.08,234.32,233.80,234.06,89256.45
min,0.00,9.14,9.14,9.15,9.12,9.14,0.00
25%,882.25,21.83,21.83,21.84,21.82,21.83,11397.25
50%,1764.50,107.69,107.69,107.92,107.41,107.69,24822.50
75%,2646.75,484.90,484.90,485.44,484.41,484.91,62871.50
max,3529.00,649.62,649.62,650.23,649.04,649.55,1312942.00


In [ ]:
df_sample.head(2)

,Unnamed: 0,Date,Ticker,Adj Close,Close,High,Low,Open,Repaired?,Volume
0,0,2023-06-27 09:30:00-04:00,ADBE,482.11,482.11,482.40,479.95,480.00,False,90397
1,1,2023-06-27 09:30:00-04:00,AMX,21.59,21.59,21.62,21.50,21.54,False,19394


In [ ]:
df_sample.info

<bound method DataFrame.info of       Unnamed: 0                       Date Ticker  Adj Close  Close   High  \
0              0  2023-06-27 09:30:00-04:00   ADBE     482.11 482.11 482.40   
1              1  2023-06-27 09:30:00-04:00    AMX      21.59  21.59  21.62   
2              2  2023-06-27 09:30:00-04:00   DDOG      93.40  93.40  94.43   
3              3  2023-06-27 09:30:00-04:00    IBM     131.15 131.15 131.26   
4              4  2023-06-27 09:30:00-04:00   INFY      15.58  15.58  15.64   
...          ...                        ...    ...        ...    ...    ...   
3525        3525  2023-07-03 12:50:00-04:00   LRCX        NaN    NaN    NaN   
3526        3526  2023-07-03 12:50:00-04:00    NOW        NaN    NaN    NaN   
3527        3527  2023-07-03 12:50:00-04:00   ORCL        NaN    NaN    NaN   
3528        3528  2023-07-03 12:50:00-04:00   RBLX        NaN    NaN    NaN   
3529        3529  2023-07-03 12:50:00-04:00    VOD        NaN    NaN    NaN   

        Low   Open 

In [ ]:
df_sample.memory_usage(deep=True)

Index            128
Unnamed: 0     28240
Date          289460
Ticker        213918
Adj Close      28240
Close          28240
High           28240
Low            28240
Open           28240
Repaired?       3530
Volume         28240
dtype: int64

In [277]:
df_sample.isnull().sum()

Unnamed: 0     0
Date           0
Ticker         0
Adj Close     10
Close         10
High          10
Low           10
Open          10
Repaired?      0
Volume         0
dtype: int64

In [294]:
df_ = df_sample[df_sample["Ticker"] =="ADBE"].copy()
df_.drop(columns=["Unnamed: 0"], inplace=True)
df_ = df_.set_index('Date')
df_.head(2)

,Ticker,Adj Close,Close,High,Low,Open,Repaired?,Volume
Date,,,,,,,,
2022-07-05,ADBE,376.49,376.49,378.52,359.82,363.50,False,3120500
2022-07-06,ADBE,382.83,382.83,385.39,374.09,378.54,False,2284600


In [ ]:
import pandas as pd
import json
import re
from datetime import datetime, timedelta
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df_.index,
                    open=df_['Open'],
                    high=df_['High'],
                    low=df_['Low'],
                    close=df_['Close'])])
fig

<font color=blue>We will continue Pandas and EDA next week</font>